In [1]:
import numpy as np
from scipy.optimize import brentq

cds_spreads = {
    3: 0.0050,  # 50 bps
    5: 0.0060,  # 60 bps
    10: 0.0100  # 100 bps
}
R = 0.60  # Recovery rate
payment_interval = 0.5  # Semiannual

def survival_probability(t, hazard_rates):
    λ1, λ2, λ3 = hazard_rates
    if t <= 3:
        return np.exp(-λ1 * t)
    elif t <= 5:
        return np.exp(-λ1 * 3 - λ2 * (t - 3))
    else:
        return np.exp(-λ1 * 3 - λ2 * 2 - λ3 * (t - 5))

def calculate_legs(T, hazard_rates):
    times = np.arange(payment_interval, T + payment_interval, payment_interval)
    prem_leg = sum(payment_interval * survival_probability(t, hazard_rates) for t in times)
    prot_leg = sum(
        survival_probability(t - payment_interval, hazard_rates) - survival_probability(t, hazard_rates)
        for t in times
    )
    return prem_leg, prot_leg

def bootstrap_hazard_rate(prev_lambdas, T, market_spread):
    def objective(new_lambda):
        trial_lambdas = prev_lambdas + [new_lambda]
        while len(trial_lambdas) < 3:
            trial_lambdas.append(1e-6)
        prem_leg, prot_leg = calculate_legs(T, trial_lambdas)
        model_spread = (1 - R) * prot_leg / prem_leg
        return model_spread - market_spread
    return brentq(objective, 1e-6, 1.0)

bootstrapped_hazard_rates = []
for T, spread in cds_spreads.items():
    λ_new = bootstrap_hazard_rate(bootstrapped_hazard_rates, T, spread)
    bootstrapped_hazard_rates.append(λ_new)

print("Bootstrapped hazard rates:")
for i, (interval, rate) in enumerate(zip([(0, 3), (3, 5), (5, 10)], bootstrapped_hazard_rates)):
    print(f"λ{i+1} for years {interval}: {rate*100:.6f}%")


Bootstrapped hazard rates:
λ1 for years (0, 3): 1.246110%
λ2 for years (3, 5): 1.881076%
λ3 for years (5, 10): 3.612631%


In [2]:
print(bootstrapped_hazard_rates)

[0.012461099501270974, 0.01881075558667068, 0.03612630830652719]


In [3]:
import numpy as np

# Bootstrapped hazard rates for each interval
lambda1 = bootstrapped_hazard_rates[0]  # for [0, 3] years
lambda2 = bootstrapped_hazard_rates[1]  # for (3, 5] years
lambda3 = bootstrapped_hazard_rates[2]   # for (5, 10] years

# Bond details
face_value = 100
annual_coupon = 3  # 3% annual
coupon_payment = annual_coupon / 2  # semiannual coupon
maturity = 7
payment_interval = 0.5
payment_times = np.arange(payment_interval, maturity + payment_interval, payment_interval)

# Define the survival probability function S(t)
def survival_probability(t):
    if t <= 3:
        return np.exp(-lambda1 * t)
    elif t <= 5:
        return np.exp(-lambda1 * 3 - lambda2 * (t - 3))
    else:
        return np.exp(-lambda1 * 3 - lambda2 * 2 - lambda3 * (t - 5))

# Compute expected present value of cash flows
bond_price = 0.0
for t in payment_times:
    cash_flow = coupon_payment
    if np.isclose(t, maturity):
        cash_flow += face_value  # add face value at maturity
    bond_price += cash_flow * survival_probability(t)

# Final result
print(f"Price of the 7-year bond: ${bond_price:.2f}")


Price of the 7-year bond: $106.08
